In [2]:
import os
import cv2
import pydicom
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from keras.models import Model

# Function to load, preprocess images, and assign labels
def load_and_preprocess_images_with_labels(directory, image_size):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

data_dir = r'D:\Alzheimers\Voting2'

# Define image sizes for each model
image_size_fmri = (75, 75)
image_size_mri = (128, 128)
image_size_pet = (128, 128)

test_data_fmri, true_labels = load_and_preprocess_images_with_labels(data_dir, image_size_fmri)
test_data_mri, _ = load_and_preprocess_images_with_labels(data_dir, image_size_mri)
test_data_pet, _ = load_and_preprocess_images_with_labels(data_dir, image_size_pet)

model_fmri = load_model('D:/Alzheimers/Models/fMRI_Task1.h5')
model_mri = load_model('D:/Alzheimers/Models/MRI_Task1.h5')
model_pet = load_model('D:/Alzheimers/Models/PET_Task1.h5')

# Get the output from the first fully connected layer of each model
fc_layer_name_fmri = 'dense'
fc_layer_name_mri = 'dense_1'
fc_layer_name_pet = 'dense_3'

model_fmri_fc = Model(inputs=model_fmri.input, outputs=model_fmri.get_layer(fc_layer_name_fmri).output)
model_mri_fc = Model(inputs=model_mri.input, outputs=model_mri.get_layer(fc_layer_name_mri).output)
model_pet_fc = Model(inputs=model_pet.input, outputs=model_pet.get_layer(fc_layer_name_pet).output)

X_fmri_features = model_fmri_fc.predict(np.repeat(test_data_fmri[:, :, :, np.newaxis], 3, axis=-1))
X_mri_features = model_mri_fc.predict(np.repeat(test_data_mri[:, :, :, np.newaxis], 3, axis=-1))
X_pet_features = model_pet_fc.predict(np.repeat(test_data_pet[:, :, :, np.newaxis], 3, axis=-1))

# Stack the features into a single array
X_combined = np.hstack((X_fmri_features, X_mri_features, X_pet_features))

# Use labels from the preprocessing step as true_labels
true_labels = true_labels

# Initialize the classifiers
logistic_reg = LogisticRegression(max_iter=5000)
svm_classifier = SVC()
random_forest = RandomForestClassifier()
gradient_boosting = GradientBoostingClassifier()

# Train the classifiers on the combined features
logistic_reg.fit(X_combined, true_labels)
svm_classifier.fit(X_combined, true_labels)
random_forest.fit(X_combined, true_labels)
gradient_boosting.fit(X_combined, true_labels)

# Make predictions with the trained classifiers
y_pred_logistic = logistic_reg.predict(X_combined)
y_pred_svm = svm_classifier.predict(X_combined)
y_pred_rf = random_forest.predict(X_combined)
y_pred_gb = gradient_boosting.predict(X_combined)

# Calculate and print accuracy for each classifier
accuracy_logistic = accuracy_score(true_labels, y_pred_logistic)
accuracy_svm = accuracy_score(true_labels, y_pred_svm)
accuracy_rf = accuracy_score(true_labels, y_pred_rf)
accuracy_gb = accuracy_score(true_labels, y_pred_gb)

print(f"Logistic Regression Accuracy: {accuracy_logistic * 100:.2f}%")
print(f"SVM Accuracy: {accuracy_svm * 100:.2f}%")
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")
print(f"Gradient Boosting Accuracy: {accuracy_gb * 100:.2f}%")


1563/1563 [==============================] - 714s 454ms/step


C:\Users\prana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 94.99%
SVM Accuracy: 65.15%
Random Forest Accuracy: 99.94%
Gradient Boosting Accuracy: 96.11%
